In [1]:
from census import Census
from api_keys import cen_key
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from us import states
from scipy.stats import linregress
from matplotlib import pyplot as plt
#all these are here in case you want to plot or what have you

In [5]:
pop_csv_path = "population_data.csv"
pop_by_state_df = pd.read_csv(pop_csv_path)
pop_by_state_df.dropna(axis=1)

,Geographic Area,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,.Alabama,"4,785,437","4,799,069","4,815,588","4,830,081","4,841,799","4,852,347","4,863,525","4,874,486","4,887,681","4,903,185"
1,.Alaska,"713,910","722,128","730,443","737,068","736,283","737,498","741,456","739,700","735,139","731,545"
2,.Arizona,"6,407,172","6,472,643","6,554,978","6,632,764","6,730,413","6,829,676","6,941,072","7,044,008","7,158,024","7,278,717"
3,.Arkansas,"2,921,964","2,940,667","2,952,164","2,959,400","2,967,392","2,978,048","2,989,918","3,001,345","3,009,733","3,017,804"
4,.California,"37,319,502","37,638,369","37,948,800","38,260,787","38,596,972","38,918,045","39,167,117","39,358,497","39,461,588","39,512,223"
5,.Colorado,"5,047,349","5,121,108","5,192,647","5,269,035","5,350,101","5,450,623","5,539,215","5,611,885","5,691,287","5,758,736"
6,.Connecticut,"3,579,114","3,588,283","3,594,547","3,594,841","3,594,524","3,587,122","3,578,141","3,573,297","3,571,520","3,565,287"
7,.Delaware,"899,593","907,381","915,179","923,576","932,487","941,252","948,921","956,823","965,479","973,764"
8,.District of Columbia,"605,226","619,800","634,924","650,581","662,328","675,400","685,815","694,906","701,547","705,749"
9,.Florida,"18,845,537","19,053,237","19,297,822","19,545,621","19,845,911","20,209,042","20,613,477","20,963,613","21,244,317","21,477,737"


In [7]:
## Below code getting population from the API. The values vary slightly from the csv but are both from census.gov
state_names = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

In [8]:
c = Census(cen_key, year=2009)
twenty09 = pd.DataFrame(c.acs5.get(("B01003_001E"), {
                         'for': 'state: 01, 02, 04, 05, 06, 08, 09, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56'})).sort_values('state')
c = Census(cen_key, year=2010)
twenty10 = pd.DataFrame(c.acs5.get(("B01003_001E"), {
                         'for': 'state: 01, 02, 04, 05, 06, 08, 09, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56'})).sort_values('state')
c = Census(cen_key, year=2011)
twenty11 = pd.DataFrame(c.acs5.get(("B01003_001E"), {
                         'for': 'state: 01, 02, 04, 05, 06, 08, 09, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56'})).sort_values('state')
c = Census(cen_key, year=2012)
twenty12 = pd.DataFrame(c.acs5.get(("B01003_001E"), {
                         'for': 'state: 01, 02, 04, 05, 06, 08, 09, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56'})).sort_values('state')
c = Census(cen_key, year=2013)
twenty13 = pd.DataFrame(c.acs5.get(("B01003_001E"), {
                         'for': 'state: 01, 02, 04, 05, 06, 08, 09, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56'})).sort_values('state')
c = Census(cen_key, year=2014)
twenty14 = pd.DataFrame(c.acs5.get(("B01003_001E"), {
                         'for': 'state: 01, 02, 04, 05, 06, 08, 09, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56'})).sort_values('state')
c = Census(cen_key, year=2015)
twenty15 = pd.DataFrame(c.acs5.get(("B01003_001E"), {
                         'for': 'state: 01, 02, 04, 05, 06, 08, 09, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56'})).sort_values('state')
c = Census(cen_key, year=2016)
twenty16 = pd.DataFrame(c.acs5.get(("B01003_001E"), {
                         'for': 'state: 01, 02, 04, 05, 06, 08, 09, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56'})).sort_values('state')
c = Census(cen_key, year=2017)
twenty17 = pd.DataFrame(c.acs5.get(("B01003_001E"), {
                         'for': 'state: 01, 02, 04, 05, 06, 08, 09, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56'})).sort_values('state')

In [9]:
twenty092 = list(twenty09['B01003_001E'])
twenty172 = list(twenty17['B01003_001E'])
#had to do this to 2009 and 2017 because the states came ordered all out of wack

In [11]:
population_df = pd.DataFrame({
    "State":state_names,
    "State Code": twenty10['state'],
    "2009": twenty092,
    "2010": twenty10['B01003_001E'],
    "2011": twenty11['B01003_001E'],
    "2012": twenty12['B01003_001E'],
    "2013": twenty13['B01003_001E'],
    "2014": twenty14['B01003_001E'],
    "2015": twenty15['B01003_001E'],
    "2016": twenty16['B01003_001E'],
    "2017": twenty172,
})

In [12]:
population_df 

,State,State Code,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Alabama,01,4633360.0,4712651.0,4747424.0,4777326.0,4799277.0,4817678.0,4830620.0,4841164.0,4850771.0
1,Alaska,02,683142.0,691189.0,700703.0,711139.0,720316.0,728300.0,733375.0,736855.0,738565.0
2,Arizona,04,6324865.0,6246816.0,6337373.0,6410979.0,6479703.0,6561516.0,6641928.0,6728577.0,6809946.0
3,Arkansas,05,2838143.0,2872684.0,2895928.0,2916372.0,2933369.0,2947036.0,2958208.0,2968472.0,2977944.0
4,California,06,36308527.0,36637290.0,36969200.0,37325068.0,37659181.0,38066920.0,38421464.0,38654206.0,38982847.0
5,Colorado,08,4843211.0,4887061.0,4966061.0,5042853.0,5119329.0,5197580.0,5278906.0,5359295.0,5436519.0
6,Connecticut,09,3494487.0,3545837.0,3558172.0,3572213.0,3583561.0,3592053.0,3593222.0,3588570.0,3594478.0
7,Delaware,10,863832.0,881278.0,890856.0,900131.0,908446.0,917060.0,926454.0,934695.0,943732.0
8,District of Columbia,11,588433.0,584400.0,593955.0,605759.0,619371.0,633736.0,647484.0,659009.0,672391.0
9,Florida,12,18222420.0,18511620.0,18688787.0,18885152.0,19091156.0,19361792.0,19645772.0,19934451.0,20278447.0
